In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import csv
import numpy as np
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk.stem
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet, stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from scipy.sparse import hstack
# Import necessary libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
import string
import re
from sklearn.preprocessing import LabelEncoder
import os
import pickle
import sys
from IPython.display import display, HTML
import urllib
import gzip
import nltk
from nltk import tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')


import numpy as np

import random
def cleanString(review):
    stopWords = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    returnString = ""
    sentence_token = tokenize.word_tokenize(review)
    idx_list = []
    for j in range(len(sentence_token)):
        #single_sentence = tokenize.word_tokenize(sentence_token[j])
        single_sentence=[lemmatizer.lemmatize(t) for t in sentence_token]
        single_sentence=[word for word in single_sentence if word.lower() not in stopWords]
        sentences_filtered = [(idx,lemmatizer.lemmatize(w.lower())) for idx,w in enumerate(single_sentence) 
                              if w.lower() not in stopWords and w.isalnum()]
        idx_list.append([x[0] for x in sentences_filtered])
        word_list = [x[1] for x in sentences_filtered]
        returnString = returnString + ' '.join(word_list) + ' '
    
    return returnString






def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  

english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer,self).build_analyzer()
        return lambda doc:(english_stemmer.stem(word) for word in analyzer(doc))
    
    
def cleanData(string1):
    articles = []
    n = 1
    for i in range(n):
        temp_string = cleanString(string1)
        articles.append(temp_string)
        print(str(i+1)+' of '+str(n)+" articles cleaned.",end='\r')
    
    return(articles)


def predict_the_message(text,file1):
    print(text)
    clntxt=cleanData(text)
    word_model = pickle.load( open( file1 + "_word_preprocessing.sav", "rb" ) )
    char_model = pickle.load( open( file1 +  "_char_preprocessing.sav", "rb" ) )
    svd_model = pickle.load( open( file1 + "_SVD_preprocessing.sav", "rb" ) )
    all_features = pickle.load( open( file1 + "train_features1_preprocessing.sav", "rb" ) )
    X_test1 = word_model.transform(clntxt)
    X_test2 = char_model.transform(clntxt)
    test_stack = hstack([X_test2,X_test1])
    test_stack=svd_model.transform(test_stack)
    #Python_corpusMLP_model
    model = pickle.load( open( file1+"MLP_model.sav", "rb" ) )
    best_thread = pairwise_distances_argmin(
            X=test_stack,
            Y=all_features,
            metric='cosine'
        )
    print(best_thread)
    y_pred_class1 = model.predict(all_features[best_thread])
    y_pred_class2 = model.predict(test_stack)
    print(y_pred_class1,y_pred_class2)
    return(y_pred_class1,y_pred_class2)

def enter(event=None):
    bt.bind('<Return>',enter)
    fr.bind('<Return>',enter)

def clicked (value):
    clearscreen()   ## to clear the screen
    label1 = Label(root, text=value, font=("Verdana", 12) )
    label1.pack()

    #Create Chat window
    ChatLog = Text(root, bd=0, bg="white", height="8", width="50", font="Arial")
    
    ChatLog.insert(END, "AASKKK:  Hello!!, Please type your question " '\n\n')
    ChatLog.config(foreground="#442265", font=("Verdana", 12, 'bold' ),borderwidth = 5)

    ChatLog.config(state=DISABLED)

    #Bind scrollbar to Chat window
    scrollbar = Scrollbar(root, command=ChatLog.yview, cursor="heart")
    ChatLog['yscrollcommand'] = scrollbar.set
    
    #Create the box to enter message
    EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial",borderwidth = 5)
    #EntryBox.bind("<Return>", send)

    #Create Button to send message
    SendButton = Button(root, font=("Verdana",12,'bold'), text="AASKKK", width="11", height=4,
                        bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',borderwidth = 5,
                        command= lambda: send(EntryBox,ChatLog,value) )

    #Place all components on the screen
    scrollbar.place(x=480,y=25, height=375)
    ChatLog.place(x=6,y=25, height=375, width=480)
    EntryBox.place(x=5, y=403, height=65, width=353)
    SendButton.place(x=360, y=403, height=65, width=135)



def clearscreen():
    list = root.pack_slaves()
    for l in list:
        l.destroy()    
        
def send(EntryBox,ChatLog,value):
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg == '':
        messagebox.showinfo("Alert", "Please enter your question and then click send")
        
    elif msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12,'bold'))

        res = chatbot_response(msg,value)
        ChatLog.insert(END, "AASKKK: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        

def chatbot_response(msg,value):
    
   # msg =cleanData(msg)  ##output is articals
    
    if value == "Python":
        file1="Python_Corpus"
        output,output2= predict_the_message(msg,file1)
        
        label=  pickle.load( open( file1+"_labelEncoder.sav", "rb" ) )
        className=label.classes_[output]
        className1=label.classes_[output2]
        print(label.classes_[output])
        print(label.classes_[output2])
        str1 = ''.join(className1)
        dict_label=  pickle.load( open( file1+"_ResponseDictionary.sav", "rb" ) )
        #dict_label[listToString(['Data Analysis'])]        
        HardCodedResult1=dict_label[listToString(label.classes_[output])]
        HardCodedResult2=dict_label[listToString(label.classes_[output2])]
        
        return("\n\n"+str(HardCodedResult1))  
    elif value == "Statistics_corpus":
        file1="statistics_Corpus"
        output,output2= predict_the_message(msg,file1)
        
        label=  pickle.load( open( file1+"_labelEncoder.sav", "rb" ) )
        className=label.classes_[output]
        className1=label.classes_[output2]
        print(label.classes_[output])
        print(label.classes_[output2])
        str1 = ''.join(className1)
        dict_label=  pickle.load( open( file1+"_ResponseDictionary.sav", "rb" ) )
        #dict_label[listToString(['Data Analysis'])]        
        HardCodedResult1=dict_label[listToString(label.classes_[output])]
        HardCodedResult2=dict_label[listToString(label.classes_[output2])]
        
        return("\n\n"+str(HardCodedResult1))  
    elif value == "Supervised Learning":
        file1="Supervised_Learning_Corpus"
        output,output2= predict_the_message(msg,file1)
        
        label=  pickle.load( open( file1+"_labelEncoder.sav", "rb" ) )
        className=label.classes_[output]
        className1=label.classes_[output2]
        print(label.classes_[output])
        print(label.classes_[output2])
        str1 = ''.join(className1)
        dict_label=  pickle.load( open( file1+"_ResponseDictionary.sav", "rb" ) )
        #dict_label[listToString(['Data Analysis'])]        
        HardCodedResult1=dict_label[listToString(label.classes_[output])]
        HardCodedResult2=dict_label[listToString(label.classes_[output2])]
        
        return("\n\n"+str(HardCodedResult1))   
    elif value == "Ensemble Techniques":
        file1="Ensemble_Techniques_Corpus"
        output,output2= predict_the_message(msg,file1)
        
        label=  pickle.load( open( file1+"_labelEncoder.sav", "rb" ) )
        className=label.classes_[output]
        className1=label.classes_[output2]
        print(label.classes_[output])
        print(label.classes_[output2])
        str1 = ''.join(className1)
        dict_label=  pickle.load( open( file1+"_ResponseDictionary.sav", "rb" ) )
        #dict_label[listToString(['Data Analysis'])]        
        HardCodedResult1=dict_label[listToString(label.classes_[output])]
        HardCodedResult2=dict_label[listToString(label.classes_[output2])]
        
        return("\n\n"+str(HardCodedResult1))  
    elif value == "Unsupervised learning":
        file1="Unsupervised_Learning_Corpus"
        output,output2= predict_the_message(msg,file1)
        
        label=  pickle.load( open( file1+"_labelEncoder.sav", "rb" ) )
        className=label.classes_[output]
        className1=label.classes_[output2]
        print(label.classes_[output])
        print(label.classes_[output2])
        str1 = ''.join(className1)
        dict_label=  pickle.load( open( file1+"_ResponseDictionary.sav", "rb" ) )
        #dict_label[listToString(['Data Analysis'])]        
        HardCodedResult1=dict_label[listToString(label.classes_[output])]
        HardCodedResult2=dict_label[listToString(label.classes_[output2])]
        
        return("\n\n"+str(HardCodedResult1))    
    else:
        file1="Combined_Corpus"
        output,output2= predict_the_message(msg,file1)
        
        label=  pickle.load( open( file1+"_labelEncoder.sav", "rb" ) )
        className=label.classes_[output]
        className1=label.classes_[output2]
        print(label.classes_[output])
        print(label.classes_[output2])
        str1 = ''.join(className1)
        dict_label=  pickle.load( open( file1+"_ResponseDictionary.sav", "rb" ) )
        #dict_label[listToString(['Data Analysis'])]        
        HardCodedResult1=dict_label[listToString(label.classes_[output])]
        HardCodedResult2=dict_label[listToString(label.classes_[output2])]

        return("\n\n"+str(HardCodedResult2))  
#Description: This is a chat bot GUI


#Import the library

from tkinter import *
from PIL import ImageTk,Image
from tkinter import messagebox

root = Tk()
#Name of the chat Bot
root.title("AASKKK") 
#Geometry of the chat Bot
root.geometry("500x600")
root.resizable(width=FALSE, height=FALSE)


# Creating Background Image
canvas=Canvas(root,width=500,height=600)
#give this image path. image should be in png format.
image=ImageTk.PhotoImage(Image.open("wall_e.png"))
canvas.create_image(0,0,anchor=NW,image=image)
canvas.place(height=600, x=0, y=0)

def restart():
    
    clearscreen()
    #Name of the chat Bot
    root.title("AASKKK") 
    #Geometry of the chat Bot
    root.geometry("500x600")
    root.resizable(width=FALSE, height=FALSE)
        
    # Creating Background Image
    canvas=Canvas(root,width=500,height=600)
    #give this image path. image should be in png format.
    image=ImageTk.PhotoImage(Image.open("wall_e.png"))
    canvas.create_image(0,0,anchor=NW,image=image)
    canvas.place(height=600, x=0, y=0)
    
    menubar= Menu(root) 
    filemenu = Menu(menubar, tearoff=0)
    menubar.add_command(label="Exit", command=root.destroy)
    menubar.add_command(label="Restart",command = lambda : restart())
    
    root.config(menu=menubar)  
    label2 = Label(root, text="Welcome", font=("Arial", 20) )
    label2.pack(padx= 20, pady =10)
    
    label2 = Label(root, text="Please select the topic from which you want to learn", font=("Arial", 15) )
    label2.pack(padx= 20, pady =20)
    
    TOPICS = [("Python                         ","Python"),
          ("Statistics                      ","Statistics"),
          ("Supervised Learning      ","Supervised Learning"),
          ("Ensemble Techniques    ","Ensemble Techniques"),
          ("Unsupervised learning    ","Unsupervised learning"),
          ("Not Sure about the topic","All Topics")
         ]
    option = StringVar()  
    option.set("Python")
    
    for text,topics in TOPICS:
        r =Radiobutton(root, text = text, variable=option , value=topics, font=("Arial", 15),bg="#49A",activebackground="#3c9d9b")
        r.pack(fill=Y,anchor = CENTER )
    
    Button2 = Button(root, text="AASKKK",fg='#ffffff',
                 bg="#32de97",activebackground="#3c9d9b",font=("Arial", 15,'bold'),padx= 30, pady =10,
                 command = lambda : clicked(option.get()))
    
    Button2.pack(padx= 50, pady =30)
    
    root.mainloop()
    
menubar= Menu(root)
filemenu = Menu(menubar, tearoff=0)
    
menubar.add_command(label="Exit", command=root.destroy)
menubar.add_command(label="Restart",command = lambda : restart())
    
root.config(menu=menubar)
    
label2 = Label(root, text="Welcome", font=("Arial", 18) )
label2.pack(padx= 20, pady =10)
    
label2 = Label(root, text="Please select the topic from which you want to learn", font=("Arial", 12) )
    
label2.pack(padx= 20, pady =20)
    
    
TOPICS = [("Python                         ","Python"),
          ("Statistics                      ","Statistics"),
          ("Supervised Learning      ","Supervised Learning"),
          ("Ensemble Techniques    ","Ensemble Techniques"),
          ("Unsupervised learning    ","Unsupervised learning"),
          ("Not Sure about the topic","All Topics")
         ]
    
option = StringVar()  
option.set("Python")
    
    
for text,topics in TOPICS:
    r =Radiobutton(root, text = text, variable=option , value=topics, font=("Arial", 15),bg="#49A",activebackground="#3c9d9b")
    r.pack(fill=Y,anchor = CENTER )
        
Button2 = Button(root, text="Submit",fg='#ffffff',
                 bg="#32de97",activebackground="#3c9d9b",font=("Arial", 15),padx= 30, pady =10,
                 command = lambda : clicked(option.get()))
    
Button2.pack(padx= 50, pady =30)
    
from sklearn.metrics.pairwise import pairwise_distances_argmin_min
    
from sklearn.metrics.pairwise import pairwise_distances_argmin
    
    
root.mainloop()

[nltk_data] Downloading package punkt to C:\Users\Kumar
[nltk_data]     Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Kumar
[nltk_data]     Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Kumar
[nltk_data]     Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Kumar
[nltk_data]     Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Kumar
[nltk_data]     Ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


linera regressioja
[334]1 articles cleaned.
[91] [56]
['formula of Linear regression']
['Linear Regression']
median
[210]1 articles cleaned.
[29] [29]
['Central Tendency and 3 Ms']
['Central Tendency and 3 Ms']
